In [ ]:
import json
from openai import OpenAI
import os
import requests

# Initialize the OpenAI client with your API key
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY")
)

# Directory where the image prompts are saved
outputs_dir = "outputs"

# List all files in the outputs directory
files = os.listdir(outputs_dir)

# Filter for image prompt files
image_prompt_files = [f for f in files if 'image_prompt' in f and f.endswith('.json')]

print(f"Found {len(image_prompt_files)} image prompt files.")

# Create a directory to save images if it doesn't exist
os.makedirs("generated_images", exist_ok=True)

# Iterate over each image prompt file
for prompt_file in image_prompt_files:
    # Construct the full path to the prompt file
    prompt_file_path = os.path.join(outputs_dir, prompt_file)

    # Load the prompt data from the JSON file
    with open(prompt_file_path, 'r', encoding='utf-8') as f:
        prompt_data = json.load(f)

    # Extract the image prompt
    image_prompt = prompt_data.get('prompt')
    final_prompt =(
        f"A high-quality, professional product photo of {image_prompt}.\n"
        f"The image should accurately represent the product's visual details.\n"
        f"Style: studio lighting, plain background, **no text**, **no logos**, **no text labels**, **no watermarks**.\n"
        f"Focus solely on the product."
    )

    if not image_prompt:
        print(f"No prompt found in {prompt_file}")
        continue

    # Print the prompt for debugging
    print(f"Using prompt from {prompt_file}:\n{final_prompt}\n")

    # Generate the image using DALL·E
    try:
        response = client.images.generate(
            model="dall-e-3",
            prompt=final_prompt,
            size='1792x1024',
            quality='hd',
            n=1,
            style='vivid'
        )
    except Exception as e:
        print(f"Error generating image for {prompt_file}: {e}")
        continue

    # Get the product name from the filename
    # Assumes the filename format is 'productname_image_prompt_timestamp.json'
    product_name = '_'.join(prompt_file.split('_')[:-2])  # Adjust based on your filename pattern
    product_name = product_name.replace('_', ' ').title()  # Format the product name nicely

    # Download and save each generated image
    for idx, image_data in enumerate(response.data):
        image_url = image_data.url
        print(f"Image {idx+1} URL: {image_url}")

        # Fetch the image content
        image_response = requests.get(image_url)
        if image_response.status_code == 200:
            # Create a filename for the image
            image_filename = f"{product_name.replace(' ', '_').lower()}_dalle_image_{idx+1}.png"
            image_path = os.path.join("generated_images", image_filename)
            # Save the image
            with open(image_path, 'wb') as f:
                f.write(image_response.content)
            print(f"Image saved as '{image_path}'")
        else:
            print(f"Failed to download image {idx+1}. HTTP Status Code: {image_response.status_code}")


    print("-" * 50)


Found 3 image prompt files.
Using prompt from breville_espresso_machine_image_prompt_20241205063108.json:
A high-quality, professional product photo of The Breville BES870XL Espresso Machine is a stainless steel, all-in-one espresso machine with an integrated grinder. It offers dose control grinding, optimal water pressure, and can go from beans to espresso in under a minute. Ideal for home baristas, it requires additional accessories for optimal use..
The image should accurately represent the product's visual details.
Style: studio lighting, plain background, **no text**, **no logos**, **no text labels**, **no watermarks**.
Focus solely on the product.

Image 1 URL: https://oaidalleapiprodscus.blob.core.windows.net/private/org-AcLuD2qoPK8CZ9AKPcHBoGIZ/user-pOSIohgrWLxPC6s1NS9fY5VD/img-Wn3AdBfNLlcyAKr4rXLgFDgl.png?st=2024-12-05T11%3A26%3A45Z&se=2024-12-05T13%3A26%3A45Z&sp=r&sv=2024-08-04&sr=b&rscd=inline&rsct=image/png&skoid=d505667d-d6c1-4a0a-bac7-5c84a87759f8&sktid=a48cca56-e6da-484e